# Chapter 4 (continued)
Starting on page 228 - Aggregating Data

## Imports

In [96]:
import numpy as np
import pandas as pd
import sqlite3

## Aggregating data

In [2]:
fb = pd.read_csv(
    '../data/fb_2018.csv', index_col='date', parse_dates=True
).assign(trading_volume=lambda df_: pd.cut(
    df_.volume, bins=3, labels=['low', 'med', 'high']
))

In [3]:
weather = pd.read_csv(
    '../data/weather_by_station.csv', index_col='date',
    parse_dates=True
)

In [4]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

### Summarizing DataFrames

In [5]:
fb.agg({
    'open': np.mean, 'high': np.max, 'low': np.min,
    'close': np.mean, 'volume': np.sum
})

open            171.45
high            218.62
low             123.02
close           171.51
volume   6949682394.00
dtype: float64

In [6]:
(weather.query('station == "GHCND:USW00094728"')
 .pivot(columns='datatype', values='value')
 [['SNOW', 'PRCP']].sum()
)

datatype
SNOW   1007.00
PRCP   1665.30
dtype: float64

In [7]:
fb.agg({
    'open': 'mean',
    'high': ['min', 'max'],
    'low': ['min', 'max'],
    'close': 'mean'
})

,open,high,low,close
mean,171.45,NaN,NaN,171.51
min,NaN,129.74,123.02,NaN
max,NaN,218.62,214.27,NaN


### Aggregating by group

In [8]:
fb.groupby('trading_volume').mean()

,open,high,low,close,volume
trading_volume,,,,,
low,171.36,173.46,169.31,171.43,24547207.71
med,175.82,179.42,172.11,175.14,79072559.12
high,167.73,170.48,161.57,168.16,141924023.33


In [9]:
(fb
 .groupby('trading_volume')
 ['close'].agg(['min', 'max', 'mean'])
)

,min,max,mean
trading_volume,,,
low,124.06,214.67,171.43
med,152.22,217.50,175.14
high,160.06,176.26,168.16


In [10]:
fb_agg = (fb
 .groupby('trading_volume')
 .agg({
     'open': 'mean', 'high': ['min', 'max'],
     'low': ['min', 'max'], 'close': 'mean'
 })
)
fb_agg

open   high           low         close
                 mean    min    max    min    max   mean
trading_volume                                          
low            171.36 129.74 216.20 123.02 212.60 171.43
med            175.82 162.85 218.62 150.75 214.27 175.14
high           167.73 161.10 180.13 149.02 173.75 168.16

In [11]:
fb_agg.loc['med', 'low']['min']

150.75

In [12]:
fb_agg.columns

MultiIndex([( 'open', 'mean'),
            ( 'high',  'min'),
            ( 'high',  'max'),
            (  'low',  'min'),
            (  'low',  'max'),
            ('close', 'mean')],
           )

In [13]:
fb_agg.columns = ['_'.join(col_agg) for col_agg in fb_agg.columns]
fb_agg.head()

,open_mean,high_min,high_max,low_min,low_max,close_mean
trading_volume,,,,,,
low,171.36,129.74,216.20,123.02,212.60,171.43
med,175.82,162.85,218.62,150.75,214.27,175.14
high,167.73,161.10,180.13,149.02,173.75,168.16


In [14]:
(weather
 .loc['2018-10']
 .query('datatype == "PRCP"')
 .groupby(level=0).mean()
 .squeeze()
 )

/var/folders/gg/0m4l7pvs5zq9d8p4tzflypdc0000gn/T/ipykernel_8937/43659499.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  .groupby(level=0).mean()


date
2018-10-01    0.01
2018-10-02    2.23
2018-10-03   19.69
2018-10-04    0.32
2018-10-05    0.97
2018-10-06    0.06
2018-10-07    0.67
2018-10-08    0.16
2018-10-09    1.04
2018-10-10    0.15
2018-10-11    3.00
2018-10-12   34.77
2018-10-13    1.90
2018-10-14    1.49
2018-10-15    1.06
2018-10-16    2.49
2018-10-17    0.03
2018-10-18    0.01
2018-10-19    0.00
2018-10-20    1.89
2018-10-21    0.54
2018-10-22    0.02
2018-10-23    0.01
2018-10-24    0.09
2018-10-25    0.00
2018-10-26    0.02
2018-10-27   18.42
2018-10-28   12.66
2018-10-29    1.76
2018-10-30    0.17
2018-10-31    0.01
Name: value, dtype: float64

In [15]:
(weather
 .query('datatype == "PRCP"')
 .groupby(['station_name', pd.Grouper(freq='Q')])
 .sum()
 .unstack()
 .sample(5, random_state=1)
 )

/var/folders/gg/0m4l7pvs5zq9d8p4tzflypdc0000gn/T/ipykernel_8937/1952782215.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  .sum()


value                                 
date                        2018-03-31 2018-06-30 2018-09-30 2018-12-31
station_name                                                           
WANTAGH 1.1 NNE, NY US          279.90     216.80     472.50     277.20
STATEN ISLAND 1.4 SE, NY US     379.40     295.30     438.80     409.90
SYOSSET 2.0 SSW, NY US          323.50     263.30     355.50     459.90
STAMFORD 4.2 S, CT US           338.00     272.10     424.70     390.00
WAYNE TWP 0.8 SSW, NJ US        246.20     295.30     620.90     422.00

In [16]:
(weather
 .query('datatype == "PRCP"')
 .groupby(level=0).mean()
 .groupby(pd.Grouper(freq='M'))
 .sum()
 .value
 .nlargest()
 )

/var/folders/gg/0m4l7pvs5zq9d8p4tzflypdc0000gn/T/ipykernel_8937/1137786562.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  .groupby(level=0).mean()


date
2018-11-30   210.59
2018-09-30   193.09
2018-08-31   192.45
2018-07-31   160.98
2018-02-28   158.11
Name: value, dtype: float64

In [17]:
(weather
 .query('datatype == "PRCP"')
 .rename(dict(value='prcp'), axis=1)
 .groupby(level=0)
 .mean()
 .groupby(pd.Grouper(freq='M'))
 .transform(np.sum)
 ['2018-01-28':'2018-02-03']
 )

/var/folders/gg/0m4l7pvs5zq9d8p4tzflypdc0000gn/T/ipykernel_8937/1354283936.py:5: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  .mean()


,prcp
date,
2018-01-28,69.31
2018-01-29,69.31
2018-01-30,69.31
2018-01-31,69.31
2018-02-01,158.11
2018-02-02,158.11
2018-02-03,158.11


In [18]:
(weather
 .query('datatype == "PRCP"')
 .rename(dict(value='prcp'), axis=1)
 .groupby(level=0).mean()
 .assign(
     total_prcp_in_month=lambda df_: df_.groupby(
         pd.Grouper(freq='M')).transform(np.sum),
     pct_monthly_prcp=lambda df_: df_.prcp.div(df_.total_prcp_in_month)
 )
 .nlargest(5, 'pct_monthly_prcp')
 )

/var/folders/gg/0m4l7pvs5zq9d8p4tzflypdc0000gn/T/ipykernel_8937/1741163927.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  .groupby(level=0).mean()


,prcp,total_prcp_in_month,pct_monthly_prcp
date,,,
2018-10-12,34.77,105.63,0.33
2018-01-13,21.66,69.31,0.31
2018-03-02,38.77,137.46,0.28
2018-04-16,39.34,140.57,0.28
2018-04-17,37.30,140.57,0.27


### Pivot tables and crosstabs

In [20]:
fb.pivot_table(columns='trading_volume')

trading_volume,low,med,high
close,171.43,175.14,168.16
high,173.46,179.42,170.48
low,169.31,172.11,161.57
open,171.36,175.82,167.73
volume,24547207.71,79072559.12,141924023.33


In [22]:
fb.pivot_table(index='trading_volume')

,close,high,low,open,volume
trading_volume,,,,,
low,171.43,173.46,169.31,171.36,24547207.71
med,175.14,179.42,172.11,175.82,79072559.12
high,168.16,170.48,161.57,167.73,141924023.33


In [25]:
weather.reset_index().pivot_table(
    index=['date', 'station', 'station_name'],
    columns='datatype',
    values='value',
    aggfunc='median'
).reset_index().tail()

datatype,date,station,station_name,AWND,DAPR,MDPR,PGTM,PRCP,SNOW,SNWD,...,WSF5,WT01,WT02,WT03,WT04,WT05,WT06,WT08,WT09,WT11
28740,2018-12-31,GHCND:USW00054787,"FARMINGDALE REPUBLIC AIRPORT, NY US",5.00,NaN,NaN,2052.00,28.70,NaN,NaN,...,15.70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28741,2018-12-31,GHCND:USW00094728,"NY CITY CENTRAL PARK, NY US",NaN,NaN,NaN,NaN,25.90,0.00,0.00,...,NaN,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28742,2018-12-31,GHCND:USW00094741,"TETERBORO AIRPORT, NJ US",1.70,NaN,NaN,1954.00,29.20,NaN,NaN,...,8.90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28743,2018-12-31,GHCND:USW00094745,"WESTCHESTER CO AIRPORT, NY US",2.70,NaN,NaN,2212.00,24.40,NaN,NaN,...,11.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28744,2018-12-31,GHCND:USW00094789,"JFK INTERNATIONAL AIRPORT, NY US",4.10,NaN,NaN,NaN,31.20,0.00,0.00,...,12.50,1.00,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
pd.crosstab(
    index=fb.trading_volume, columns=fb.index.month,
    colnames=['month']
)

month,1,2,3,4,5,6,7,8,9,10,11,12
trading_volume,,,,,,,,,,,,
low,20,19,15,20,22,21,18,23,19,23,21,19
med,1,0,4,1,0,0,2,0,0,0,0,0
high,0,0,2,0,0,0,1,0,0,0,0,0


In [27]:
pd.crosstab(
    index=fb.trading_volume, columns=fb.index.month,
    colnames=['month'], values=fb.close, aggfunc=np.mean
)

month,1,2,3,4,5,6,7,8,9,10,11,12
trading_volume,,,,,,,,,,,,
low,185.24,180.27,177.07,163.29,182.93,195.27,201.92,177.49,164.38,154.19,141.64,137.16
med,179.37,NaN,164.76,174.16,NaN,NaN,194.28,NaN,NaN,NaN,NaN,NaN
high,NaN,NaN,164.11,NaN,NaN,NaN,176.26,NaN,NaN,NaN,NaN,NaN


In [28]:
snow_data = weather.query('datatype == "SNOW"')

In [29]:
pd.crosstab(
    index=snow_data.station_name,
    columns=snow_data.index.month,
    colnames=['month'],
    values=snow_data.value,
    aggfunc=lambda x: (x > 0).sum(),
    margins=True,
    margins_name='total observations of snow'
)

month,1,2,3,4,5,6,7,8,9,10,11,12,total observations of snow
station_name,,,,,,,,,,,,,
"ALBERTSON 0.2 SSE, NY US",3.00,1.00,3.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,9
"AMITYVILLE 0.1 WSW, NY US",1.00,0.00,1.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3
"AMITYVILLE 0.6 NNE, NY US",3.00,1.00,3.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,8
"ARMONK 0.3 SE, NY US",6.00,4.00,6.00,3.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,3.00,23
"BLOOMINGDALE 0.7 SSE, NJ US",2.00,1.00,3.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
"WESTFIELD 0.6 NE, NJ US",3.00,0.00,4.00,1.00,0.00,NaN,0.00,0.00,0.00,NaN,1.00,NaN,9
"WOODBRIDGE TWP 1.1 ESE, NJ US",4.00,1.00,3.00,2.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,11
"WOODBRIDGE TWP 1.1 NNE, NJ US",2.00,1.00,3.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,7


## Working with time series data

In [30]:
fb = pd.read_csv(
    '../data/fb_2018.csv', index_col='date', parse_dates=True
).assign(trading_volume=lambda x: pd.cut(
    x.volume, bins=3, labels=['low', 'med', 'high']
))

### Time-based selection and filtering

In [33]:
fb['2018-10-11':'2018-10-15']

,open,high,low,close,volume,trading_volume
date,,,,,,
2018-10-11,150.13,154.81,149.16,153.35,35338901,low
2018-10-12,156.73,156.89,151.30,153.74,25293492,low
2018-10-15,153.32,155.57,152.55,153.52,15433521,low


In [35]:
fb.loc['2018-q1'].equals(fb['2018-01':'2018-03'])

True

In [36]:
fb.first('1W')

,open,high,low,close,volume,trading_volume
date,,,,,,
2018-01-02,177.68,181.58,177.55,181.42,18151903,low
2018-01-03,181.88,184.78,181.33,184.67,16886563,low
2018-01-04,184.90,186.21,184.10,184.33,13880896,low
2018-01-05,185.59,186.90,184.93,186.85,13574535,low


In [37]:
fb.last('1W')

,open,high,low,close,volume,trading_volume
date,,,,,,
2018-12-31,134.45,134.64,129.95,131.09,24625308,low


In [38]:
fb_reindexed = fb.reindex(
    pd.date_range('2018-01-01', '2018-12-31', freq='D')
)

In [43]:
fb_reindexed.first('1D').isna().squeeze().all()

True

In [44]:
fb_reindexed.loc['2018-Q1'].first_valid_index()

Timestamp('2018-01-02 00:00:00', freq='D')

In [46]:
fb_reindexed.loc['2018-q1'].last_valid_index()

Timestamp('2018-03-29 00:00:00', freq='D')

In [47]:
fb_reindexed.asof('2018-03-31')

open                  155.15
high                  161.42
low                   154.14
close                 159.79
volume           59434293.00
trading_volume           low
Name: 2018-03-31 00:00:00, dtype: object

In [51]:
pd.read_csv('../data/fb_week_of_may_20_per_minute.csv')

,date,open,high,low,close,volume
0,2019-05-20 09-30,181.62,181.62,181.62,181.62,159049.00
1,2019-05-20 09-31,182.61,182.61,182.61,182.61,468017.00
2,2019-05-20 09-32,182.75,182.75,182.75,182.75,97258.00
3,2019-05-20 09-33,182.95,182.95,182.95,182.95,43961.00
4,2019-05-20 09-34,183.06,183.06,183.06,183.06,79562.00
...,...,...,...,...,...,...
1946,2019-05-24 15-56,181.25,181.25,181.25,181.25,22149.00
1947,2019-05-24 15-57,181.20,181.20,181.20,181.20,23860.00
1948,2019-05-24 15-58,181.04,181.04,181.04,181.04,57547.00
1949,2019-05-24 15-59,181.07,181.07,181.07,181.07,52994.00


In [48]:
stock_data_per_minute = pd.read_csv(
    '../data/fb_week_of_may_20_per_minute.csv',
    index_col='date', parse_dates=True,
    date_parser=lambda x: pd.to_datetime(x, format='%Y-%m-%d %H-%M')
)

In [49]:
stock_data_per_minute.head()

,open,high,low,close,volume
date,,,,,
2019-05-20 09:30:00,181.62,181.62,181.62,181.62,159049.00
2019-05-20 09:31:00,182.61,182.61,182.61,182.61,468017.00
2019-05-20 09:32:00,182.75,182.75,182.75,182.75,97258.00
2019-05-20 09:33:00,182.95,182.95,182.95,182.95,43961.00
2019-05-20 09:34:00,183.06,183.06,183.06,183.06,79562.00


In [52]:
stock_data_per_minute.groupby(pd.Grouper(freq='1D')).agg({
    'open': 'first',
    'high': 'max',
    'low': 'min',
    'close': 'last',
    'volume': 'sum'
})

,open,high,low,close,volume
date,,,,,
2019-05-20,181.62,184.18,181.62,182.72,10044838.00
2019-05-21,184.53,185.58,183.97,184.82,7198405.00
2019-05-22,184.81,186.56,184.01,185.32,8412433.00
2019-05-23,182.50,183.73,179.76,180.87,12479171.00
2019-05-24,182.33,183.52,181.04,181.06,7686030.00


In [53]:
stock_data_per_minute.at_time('9:30')

,open,high,low,close,volume
date,,,,,
2019-05-20 09:30:00,181.62,181.62,181.62,181.62,159049.00
2019-05-21 09:30:00,184.53,184.53,184.53,184.53,58171.00
2019-05-22 09:30:00,184.81,184.81,184.81,184.81,41585.00
2019-05-23 09:30:00,182.50,182.50,182.50,182.50,121930.00
2019-05-24 09:30:00,182.33,182.33,182.33,182.33,52681.00


In [54]:
stock_data_per_minute.between_time('15:59', '16:00')

,open,high,low,close,volume
date,,,,,
2019-05-20 15:59:00,182.91,182.91,182.91,182.91,134569.00
2019-05-20 16:00:00,182.72,182.72,182.72,182.72,1113672.00
2019-05-21 15:59:00,184.84,184.84,184.84,184.84,61606.00
2019-05-21 16:00:00,184.82,184.82,184.82,184.82,801080.00
2019-05-22 15:59:00,185.29,185.29,185.29,185.29,96099.00
2019-05-22 16:00:00,185.32,185.32,185.32,185.32,1220993.00
2019-05-23 15:59:00,180.72,180.72,180.72,180.72,109648.00
2019-05-23 16:00:00,180.87,180.87,180.87,180.87,1329217.00
2019-05-24 15:59:00,181.07,181.07,181.07,181.07,52994.00


In [63]:
shares_traded_in_first_30_min = (stock_data_per_minute
 .between_time('9:30', '10:00')
 .groupby(pd.Grouper(freq='1D'))
 .filter(lambda x: (x.volume > 0).all())
 .volume.mean()
)
shares_traded_in_first_30_min

64934.25806451613

In [64]:
shares_traded_in_last_30_min = (stock_data_per_minute
 .between_time('15:30', '16:00')
 .groupby(pd.Grouper(freq='1D'))
 .filter(lambda x: (x.volume > 0).all())
 .volume.mean()
)
shares_traded_in_last_30_min

46341.290322580644

In [65]:
shares_traded_in_first_30_min - shares_traded_in_last_30_min

18592.967741935485

### Shifting for lagged data

In [68]:
fb.assign(
    prior_close=lambda df_: df_.close.shift(),
    after_hours_change_in_price=lambda df_: df_.open - df_.prior_close,
    abs_change=lambda df_: df_.after_hours_change_in_price.abs()
).nlargest(5, 'abs_change')

,open,high,low,close,volume,trading_volume,prior_close,after_hours_change_in_price,abs_change
date,,,,,,,,,
2018-07-26,174.89,180.13,173.75,176.26,169803668,high,217.50,-42.61,42.61
2018-04-26,173.22,176.27,170.80,174.16,77556934,med,159.69,13.53,13.53
2018-01-12,178.06,181.48,177.40,179.37,77551299,med,187.77,-9.71,9.71
2018-10-31,155.00,156.40,148.96,151.79,60101251,low,146.22,8.78,8.78
2018-03-19,177.01,177.17,170.06,172.56,88140060,med,185.09,-8.08,8.08


### Differenced data

In [70]:
(fb.drop(columns='trading_volume') - fb.drop(columns='trading_volume').shift()) \
    .equals(fb.drop(columns='trading_volume').diff())

True

In [71]:
fb.drop(columns='trading_volume').diff().head()

,open,high,low,close,volume
date,,,,,
2018-01-02,NaN,NaN,NaN,NaN,NaN
2018-01-03,4.20,3.20,3.78,3.25,-1265340.00
2018-01-04,3.02,1.43,2.77,-0.34,-3005667.00
2018-01-05,0.69,0.69,0.83,2.52,-306361.00
2018-01-08,1.61,2.00,1.40,1.43,4420191.00


In [72]:
fb

,open,high,low,close,volume,trading_volume
date,,,,,,
2018-01-02,177.68,181.58,177.55,181.42,18151903,low
2018-01-03,181.88,184.78,181.33,184.67,16886563,low
2018-01-04,184.90,186.21,184.10,184.33,13880896,low
2018-01-05,185.59,186.90,184.93,186.85,13574535,low
2018-01-08,187.20,188.90,186.33,188.28,17994726,low
...,...,...,...,...,...,...
2018-12-24,123.10,129.74,123.02,124.06,22066002,low
2018-12-26,126.00,134.24,125.89,134.18,39723370,low
2018-12-27,132.44,134.99,129.67,134.52,31202509,low


In [74]:
stock_data_per_minute.resample('1D').agg({
    'open': 'first',
    'high': 'max',
    'low': 'min',
    'close': 'last',
    'volume': 'sum'
})

,open,high,low,close,volume
date,,,,,
2019-05-20,181.62,184.18,181.62,182.72,10044838.00
2019-05-21,184.53,185.58,183.97,184.82,7198405.00
2019-05-22,184.81,186.56,184.01,185.32,8412433.00
2019-05-23,182.50,183.73,179.76,180.87,12479171.00
2019-05-24,182.33,183.52,181.04,181.06,7686030.00


In [75]:
fb.resample('Q').mean()

/var/folders/gg/0m4l7pvs5zq9d8p4tzflypdc0000gn/T/ipykernel_8937/46532002.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  fb.resample('Q').mean()


,open,high,low,close,volume
date,,,,,
2018-03-31,179.47,181.79,177.04,179.55,32926396.70
2018-06-30,180.37,182.28,178.60,180.70,24055317.75
2018-09-30,180.81,182.89,178.96,181.03,27019824.76
2018-12-31,145.27,147.62,142.72,144.87,26974331.73


In [94]:
(fb
 .drop(columns='trading_volume')
 .resample('Q')
 .apply(lambda x: x.last('1D') - x.first('1D').values)
#  .apply(lambda x: x.last('1D').values - x.first('1D').values)
)

,open,high,low,close,volume
date,,,,,
2018-03-31,-22.53,-20.16,-23.41,-21.63,41282390
2018-06-30,39.51,38.40,39.84,38.93,-20984389
2018-09-30,-25.04,-28.66,-29.66,-32.90,20304060
2018-12-31,-28.58,-31.24,-31.31,-31.35,-1782369


In [91]:
melted_stock_data = pd.read_csv(
    '../data/melted_stock_data.csv',
    index_col='date', parse_dates=True
)
melted_stock_data.head()

,price
date,
2019-05-20 09:30:00,181.62
2019-05-20 09:31:00,182.61
2019-05-20 09:32:00,182.75
2019-05-20 09:33:00,182.95
2019-05-20 09:34:00,183.06


In [92]:
melted_stock_data.resample('1D').ohlc()['price']

,open,high,low,close
date,,,,
2019-05-20,181.62,184.18,181.62,182.72
2019-05-21,184.53,185.58,183.97,184.82
2019-05-22,184.81,186.56,184.01,185.32
2019-05-23,182.50,183.73,179.76,180.87
2019-05-24,182.33,183.52,181.04,181.06


In [95]:
fb.resample('6H').asfreq().head()

,open,high,low,close,volume,trading_volume
date,,,,,,
2018-01-02 00:00:00,177.68,181.58,177.55,181.42,18151903.00,low
2018-01-02 06:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-02 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-02 18:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-03 00:00:00,181.88,184.78,181.33,184.67,16886563.00,low


### Merging time series

In [98]:
with sqlite3.connect('../data/stocks.db') as connection:
    fb_prices = pd.read_sql(
        'SELECT * FROM fb_prices', connection,
        index_col='date', parse_dates=['date']
    )
    aapl_prices = pd.read_sql(
        'SELECT * FROM aapl_prices', connection,
        index_col='date', parse_dates=['date']
    )

In [99]:
fb_prices.index.second.unique()

Int64Index([0], dtype='int64', name='date')

In [100]:
aapl_prices.index.second.unique()

Int64Index([ 0, 52, 36, 34, 55, 35,  7, 12, 59, 17,  5, 20, 26, 23, 54, 49, 19,
            53, 11, 22, 13, 21, 10, 46, 42, 38, 33, 18, 16,  9, 56, 39,  2, 50,
            31, 58, 48, 24, 29,  6, 47, 51, 40,  3, 15, 14, 25,  4, 43,  8, 32,
            27, 30, 45,  1, 44, 57, 41, 37, 28],
           dtype='int64', name='date')

In [102]:
pd.merge_asof(
    fb_prices, aapl_prices,
    left_index=True, right_index=True,
    # merge with nearest minute
    direction='nearest',
    tolerance=pd.Timedelta(30, unit='s')
).head()

,FB,AAPL
date,,
2019-05-20 09:30:00,181.62,183.52
2019-05-20 09:31:00,182.61,NaN
2019-05-20 09:32:00,182.75,182.87
2019-05-20 09:33:00,182.95,182.50
2019-05-20 09:34:00,183.06,182.11


In [103]:
pd.merge_ordered(
    fb_prices.reset_index(), aapl_prices.reset_index()
).set_index('date').head()

,FB,AAPL
date,,
2019-05-20 09:30:00,181.62,183.52
2019-05-20 09:31:00,182.61,NaN
2019-05-20 09:31:52,NaN,182.87
2019-05-20 09:32:00,182.75,NaN
2019-05-20 09:32:36,NaN,182.50
